## 주식 가격 (KOSPI 지수)

### 과거 데이터 일괄로 가져와서 저장

In [26]:
# 네이버 금융
import FinanceDataReader as fdr

index_code = 'KS11'     # KOSPI
start_date = '2010-12-24'       # 20101224
end_date = '2023-09-03'

idx_price = fdr.DataReader(index_code, start=start_date, end=end_date)
# print(idx_price.head())
# print(idx_price.tail())

In [27]:
df_idx_price = idx_price.reset_index()
df_idx_price.shape

(3126, 7)

In [28]:
# 결측치 데이터 개수 확인 (by 컬럼)
df_idx_price.isnull().sum()

Date         0
Open         2
High         2
Low          2
Close        2
Adj Close    2
Volume       2
dtype: int64

In [29]:
# 종가 데이터의 결측치 확인
df_idx_price[df_idx_price['Close'].isnull()]

,Date,Open,High,Low,Close,Adj Close,Volume
190,2011-10-04,NaN,NaN,NaN,NaN,NaN,NaN
1147,2015-08-14,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# 종가 데이터의 결측치 데이터 행 삭제
df_idx_price.dropna(subset = ['Close'], inplace=True)

In [31]:
df_idx_price.shape

(3124, 7)

In [32]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

# Date 형식 변환 => 문자열 yyyymmdd
df_idx_price['Date'] = df_idx_price['Date'].astype('str')
df_idx_price['Date'] = df_idx_price['Date'].str.replace('-', '')
print(f'{index_code}, {df_idx_price.shape[0]}건의 데이터를 저장합니다.')
# 수정종가(Adj. Close) 저장하지 않음. 
for row in df_idx_price.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume) \
                            values (%s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (index_code, row[1], row[2], row[3], row[4], row[5], row[7]))
conn.commit()
print(f'{index_code}, {df_idx_price.shape[0]}건의 데이터 저장 완료!')
conn.close()

KS11, 3124건의 데이터를 저장합니다.
KS11, 3124건의 데이터 저장 완료!
